## Manim - Extra

Agregamos en esta notebook un par de elementos útiles que no llegamos a comentar en las clases.

In [ ]:
from manim import *

### AnimationGroup y variantes

Una manera de enlazar varios movimientos en una sola animación es `AnimationGroup` que, como su nombre lo indica, permite agrupar varias animaciones para realizar simultáneamente (o casi). Veamos un ejemplo. 

### Primera Animación

In [ ]:
%%manim -qm CEA00

class CEA00(Scene):
    def construct(self):
        colores   = [BLUE,TEAL,GREEN,YELLOW,GOLD,RED,MAROON,PINK,PURPLE]
        cuads     = VGroup(*[Square(color=colores[i]) for i in range(9)]).arrange_in_grid(3,3)
        anim_list = []
        for i in range(9):
            anim_list.append(Create(cuads[i]))
        anim      = AnimationGroup(*anim_list)
        self.play(anim)
        self.wait()

#### Análisis

La idea es muy simple. El código tiene más trucos de Python que específicos de Manim. 

Generamos una lista con varios colores. Luego definimos un `VGroup` formado por cuadrados. Para ello, en primer lugar generamos una lista por comprensión: 

    [Square(color=colores[i]) for i in range(9)]

es una lista de 9 cuadrados, cada uno con un color distinto, tomado de `colores` (es importante que `colores` tenga al menos 9 colores guardados). La sintaxis completa: 

    VGroup(*[Square(color=colores[i]) for i in range(9)])

corresponde a *desempaquetar* la lista (eso lo hace el `*` adelante) y con los elementos de la lista generar un `VGroup`. A ese `VGroup` le aplicamos la función `arrange_in_grid` que distribuye el contenido en una grilla, en este caso, de $3\times 3$. 

Lo que queremos es animar la creación simultánea de cada cuadrado. Para eso definimos una nueva lista `anim_list` cuyo contenido serán las animaciones de creación de cada cuadrado. Le vamos agregando estos *elementos*  a la lista dentro de un `for`. También podríamos haber definido está lista por comprensión, de manera similar a como definimos la lista de cuadrados. 

Finalmente, construimos un `AnimationGroup` con el contenido de nuestra lista de animaciones (de nuevo el `*`). `AnimationGroup` es en realidad una animación (formada por muchas otras) de modo que podemos correrla dentro de `play`. Queda lindo, ¿no? 

### Segunda Animación

Alguien podría pensar en hacer esta animación *a mano*, sin `AnimationGroup`. Esencialmente, podríamos tener el grupo de cuadrados y poner: 

    self.play(Create(cuads[0]),Create(cuads[1]),...)

Eso tiene poco vuelo por dos razones. La primera es que no permite multiplicar el movimiento, como ocurre en la siguiente animación: 

In [ ]:
%%manim -qm CEA01

class CEA01(Scene):
    def construct(self):
        colores   = [BLUE,TEAL,GREEN,YELLOW,GOLD,RED,MAROON,PURPLE,PINK]
        cuads     = VGroup(*[Square(color=colores[i%9]).scale(0.4) for i in range(45)]).arrange_in_grid(5,9)
        anim_list = []
        for i in range(45):
            anim_list.append(Create(cuads[i]))
        anim      = AnimationGroup(*anim_list)
        self.play(anim)
        self.wait()

### Tercera Animación: lag_ratio

La segunda razón por la que `AnimationGroup` viene bien es que permite generar un pequeño delay en la ejecución de las animaciones. Esto se logra con el parámetro `lag_ratio`. Por defecto, el `lag_ratio` de `AnimationGroup` es 0: esto significa que todas las animaciones comienzan simultáneamente. Si el `lag_ratio` es 0.5 entonces cada animación comenzará cuando la anterior esté por la mitad. Veamos un ejemplo: 

In [ ]:
%%manim -qm CEA02

class CEA02(Scene):
    def construct(self):
        colores   = [BLUE,TEAL,GREEN,YELLOW,GOLD,RED,MAROON,PURPLE,PINK]
        cuads     = VGroup(*[Square(color=colores[i]) for i in range(9)]).arrange_in_grid(3,3)
        anim_list = []
        for i in range(9):
            anim_list.append(Create(cuads[i]))
        anim      = AnimationGroup(*anim_list,lag_ratio=0.3)
        self.play(anim)
        self.wait()

#### Análisis

La aplicación de `lag_ratio=0.3` a nuestro `AnimationGroup` hace que la creación de cada cuadrado comience cuando la creación del cuadrado anterior va por el 30%. 

### Cuarta Animación

Existen dos variantes a `AnimationGroup`. Una es `LaggedStart` y la otra es `Succession`.  `LaggedStart` es equivalente a `AnimationGroup` con un `lag_ratio` apenas mayor que 0, como para que se note cierto retraso entre una animación y otra. `Succession` es equivalente a `AnimationGroup` con `lag_ratio`=1 (es decir: es una sucesión de animaciones, en la que cada una comienza justo cuando la anterior terminó). Hagamos un ejemplo combinando estas opciones:

In [ ]:
%%manim -qm CEA04

class CEA04(Scene):
    def construct(self):
        colores   = [BLUE,TEAL,GREEN,YELLOW,GOLD,RED,MAROON,PINK,PURPLE]
        cuads     = VGroup(*[Square(color=colores[i%9]).scale(0.4) for i in range(45)]).arrange_in_grid(5,9)
        anim_list = []
        for i in range(5):
            fila = [Create(cuads[9*i+j]) for j in range(9)]
            anim_list.append(LaggedStart(*fila))
        anim_total = Succession(*anim_list)
        self.play(anim_total)
        self.wait()

#### Análisis

Lo más interesante es observar que estamos armando una `Succession` de animaciones, cada una de las cuales es una `LaggedStart`. Las `LaggedStart`s corresponden a las creaciones de cada fila, mientras que la `Succession` hilvana una fila tras la otra. Naturalmente, se pueden anidar cantidades arbitrarias de `AnimationGroup`s, `LaggedStart`s y `Succession`s. 

**<span style="color:TEAL">Ejercicio 1:</span>** Implementar una animación en la que aparezcan 20 puntos en línea. Los puntos deben "caer" (moverse una unidad en sentido `DOWN`), uno tras otro, con un pequeño delay. 

### Caminos

Otro recurso muy práctico consiste en trazar el recorrido de un punto (como si fuera un lápiz escribiendo). Más allá del recurso en sí, la técnica es interesante porque nos permite introducir varios elementos que pueden resultar útiles en sí mismos. Veamos un ejemplo:

### Quinta Animación

In [ ]:
%%manim -qm CEA04

class CEA04(Scene):
    def construct(self):
        f    = lambda t: np.array((2*np.sin(t), np.sin(2 * t),0))
        func = ParametricFunction(f, t_range = np.array([0,TAU]), fill_opacity=0)
        dot  = Dot(color=BLUE).move_to(f(0))
        
        path = VMobject(color = BLUE)
        path.set_points_as_corners([dot.get_center(), dot.get_center()])
        def update_path(m):
            m.add_points_as_corners([dot.get_center()])
            
        path.add_updater(update_path)
        
        self.play(Create(dot))
        self.add(path)
        self.play(MoveAlongPath(dot,func),rate_func=linear,run_time=3)
        self.play(FadeOut(dot))
        self.wait()

#### Análisis

El primer elemento de interés es `ParametricFunction`. Esta clase nos permite definir el gráfico de una función paramétrica. En este caso, tenemos una función que depende de $t$ y se la pasamos al constructor de `ParametricFunction` junto con el rango de valores en el que debe tomarse $t$. En este caso no queremos mostrar direcamente el gráfico resultante (podríamos hacerlo con `self.play(Create(func))`), sino que queremos usar esa trayectoria como una guía para mover un punto. Para ello creamos el punto `dot` y creamos también un `VMobject` (Mobject vectorizado), en principio vacío pero de color azul, llamado `path`. A continuación utilizamos la función, propia de cualquier `VMobject` `set_points_as_corners()` que nos permite agregar puntos al `VMobjects`. Estos puntos deben ir dentro de una lista y son interpretados como vértices. Es decir que si le pasamos a `set_points_as_corners()` una lista con varios puntos el `VMobject` se va a convertir automáticamente en la poligonal que une esos puntos (en el orden en que se los demos). En este caso, agregamos un único punto dado por el centro de `dot`, que a su vez está posicionado en el comienzo de la curva paramétrica. 

Luego, definimos la función `update_path` que cargaremos como un *updater* para `path`. Esta función simplemente *agrega* nuevos puntos como vértices al Mobject (observar que aplicamos `add_points_as_corners()`, en lugar de `set_points_as_corners()`, como hicimos cuando `path` estaba vacío). El punto agregado es nuevamente el centro de `dot`. 

Finalmente, animamos: creamos el punto y agregamos `path`. Esto en principio no agrega nada porque `path` está formado por un único punto (en el centro de `dot`). A continuación hacemos la animación importante: 

    self.play(MoveAlongPath(func,dot),rate_func=linear,run_time=5)

`MoveAlongPath` es una animación que recibe como datos un camino (por ejemplo, una `ParametricFunction`, pero podría ser también el gráfico de una función definida sobre un par de ejes coordenados) y un Mobject que será el que se moverá. En este caso, movemos `dot` a lo largo de `func`. Lo que ocurre es que `path` tiene nuestro *updater*  que le va agregando como nuevo vértice el punto central de `dot`. Además, `path` está incorporado a la escena. Por lo tanto, a medida que `dot` realiza su viaje a través de `func`, `path` va creciendo delante de nuestros ojos. Por último, eliminamos `dot` de la pantalla, dejando sólo `path`.

Observaciones:
* `func` es el gráfico de la función paramétrica. Podríamos dibujarlo directamente haciendo `self.play(Create(func))`.
* Es importante observar que para obtener `func` no necesitamos un objeto de clase `Axes`. Graficamos directamente sobre la pantalla y las coordenadas de los puntos del gráfico están en unidades de pantalla. Así como existe `ParametricFunction` también disponemos de funciones: `FunctionGraph` e `ImplicitFunction`. En todos los casos el gráfico se define en unidades de pantalla y se puede mostrar directamente, sin ejes. 
* La suavidad del dibujo depende de `run_time`. `run_time` determina la cantidad de cuadros que va a consumir la animación (por defecto, 30 cuadros por segundo). Es decir que si se deja `run_time` en su valor por defecto (1), todo el recorrido de `dot` debe completarse en 30 cuadros. Dicho de otro modo, la curva se fraccionará en 30 segmentos que serán lo que veremos como resultado. Si el tiempo es muy corto para la complejidad de la curva, se notaran los segmentos y la curva no lucirá suave. Aumentando el `run_time` aumentamos el número de cuadros y así la densidad de *vértices* de la poligonal que forma nuestra curva. Los invito a poner `run_time=1` y constatar que se grafican exactamente 30 segmentos. 
* Esta animación podría hacerse sin `MoveAlongPath`: definiendo un `ValueTracker` que represente el parámetro $t$ y poniendo un *updater* para `dot` que corrija su posición según las coordenadas de $f(t)$. Ciertamente `MoveAlongPath` es más cómodo.

### Sexta Animación

En este último ejemplo mostramos otro mecanismo para producir el mismo efecto.

In [ ]:
%%manim -qm CEA05

class CEA05(Scene):
    def construct(self):
        f    = lambda t: np.sin(t)
        x    = ValueTracker(-2*PI)
        dot  = always_redraw(lambda: Dot(color=BLUE).move_to(RIGHT*x.get_value()+UP*f(x.get_value())))
        
        path = VMobject()
        def update_path(m):
            m.become(FunctionGraph(f,x_range=[-2*PI,x.get_value()],color=BLUE))
            
        path.add_updater(update_path)
        
        self.play(FadeIn(dot))
        self.add(path)
        self.play(x.animate.set_value(2*PI),run_time=2)
        self.wait()

#### Análisis

En este caso utilizamos un `ValueTracker` que representa la coordenada $x$ del punto, que definimos dentro de un `always_redraw`. Para actualizar el `path` en lugar de ir agregándole puntos utilizamos la función `become()` propia de todo Mobject. `become` hace que el Mobject se transforme en el parámetro. En este caso, le pasamos a `become` una `FunctionGraph` que grafica la función seno en un rango de $x$ que va desde $-2\pi$ (la posición inicial del punto) hasta el valor del `ValueTracker`. Para la animación basta con actualizar el valor de `x`: el punto se redibuja automáticamente en la posición deseada y el `path` se convierte en una porción cada vez mayor del gráfico. 

**<span style="color:TEAL">Ejercicio 2:</span>** Implementar una animación en la que se dibujen simultáneamente tres curvas: $\cos(x)$, $\cos(x+\frac{\pi}{2})$ y $\cos(x+\pi)$. Deben aparecer inicialmente tres puntos (de colores distintos) y deben moverse todos al mismo tiempo trazando cada una de las curvas. 